In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from keras.models import model_from_json
from datetime import datetime
from keras.applications import inception_v3 as inc_net
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D#, GlobalMaxPooling2D 
from keras.models import load_model
from keras.callbacksTest import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
import os

Using TensorFlow backend.


In [2]:
img_size = 299
my_epochs_basic = 3
my_epochs_retrain = 12
is_shuffle = True
write_batch_per = True

data_type = 'mushrooms24'
if(data_type == 'mushrooms128'):
    num_classes = 128
    my_batch_size = 128
    my_steps_per_epoch = 850
    my_val_steps = 18
elif(data_type == 'mushrooms24'):
    num_classes = 24
    my_batch_size = 64
    my_steps_per_epoch = 376
    my_val_steps = 18
elif(data_type == 'flowers'):
    num_classes = 5
    my_batch_size = 16
    my_steps_per_epoch = 188
    my_val_steps = 42
data_path_train = os.path.join('data',data_type,'train')
data_path_test = os.path.join('data',data_type,'test')

In [3]:
# idk needs a loot of improvement, but just for testing
base_model = inc_net.InceptionV3(weights='imagenet', include_top=False)
print("InceptionV3 loaded")
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#freezing all layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

InceptionV3 loaded


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range = 0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set= train_datagen.flow_from_directory(data_path_train,
                                                target_size = (img_size, img_size),
                                                batch_size=my_batch_size,
                                                class_mode='categorical')
val_set = test_datagen.flow_from_directory(data_path_test,
                                           target_size= (img_size, img_size),
                                           batch_size=my_batch_size,
                                           class_mode='categorical')

Found 24067 images belonging to 24 classes.
Found 1274 images belonging to 24 classes.


In [5]:
time_of_test = str(datetime.now().strftime("%d-%m-%y %H:%M"))
file_name= time_of_test+'_Epochs'+str(my_epochs_basic)+'_Steps'+str(my_steps_per_epoch)+'_ImageSize'+str(img_size)+'_BatchSize'+str(my_batch_size)

log_path = os.path.join('logs',data_type,'pretrain','')
model_path = os.path.join('trainedModels',data_type,'pretrain','')
if not os.path.exists(log_path):
    os.makedirs(log_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.fit_generator(training_set,
                    steps_per_epoch=my_steps_per_epoch,
                    epochs=my_epochs_basic,
                    validation_data=val_set,
                    validation_steps=my_val_steps,
                    shuffle=is_shuffle,
                    callbacks=[
                        TensorBoard(
                            log_dir=log_path,
                            write_batch_performance=write_batch_per)]
                   )

Epoch 1/3
376/376 [==============================] - 749s 2s/step - loss: 2.2052 - acc: 0.3748 - val_loss: 1.9689 - val_acc: 0.4141
Epoch 2/3
376/376 [==============================] - 660s 2s/step - loss: 1.5036 - acc: 0.5448 - val_loss: 2.7374 - val_acc: 0.3238
Epoch 3/3
376/376 [==============================] - 571s 2s/step - loss: 1.3048 - acc: 0.6006 - val_loss: 2.1553 - val_acc: 0.4340


In [6]:
# SAVE THE CURRENT MODEL
print(model_path+file_name+'.h5')
model.save(model_path+file_name+'.h5')  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk as> "+file_name)

trainedModels/mushrooms24/pretrain/05-07-18 21:27_Epochs3_Steps376_ImageSize299_BatchSize64.h5
Saved model to disk as> 05-07-18 21:27_Epochs3_Steps376_ImageSize299_BatchSize64


In [17]:
# LOAD THE CURRENT MODEL
#file_name= '05-07-18 21:27_Epochs3_Steps376_ImageSize299_BatchSize64'
model = load_model(model_path+file_name+'.h5')
print('Model loaded> '+file_name)

Model loaded> 05-07-18 21:27_Epochs3_Steps376_ImageSize299_BatchSize64


In [18]:
#tbCallBack_finetune = TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

#model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True), loss='categorical_crossentropy', metrics=['accuracy'] )

In [19]:
time_of_real = str(datetime.now().strftime("%d-%m-%y %H:%M"))
file_name = time_of_real+'__Epochs'+str(my_epochs_retrain)+'_Steps'+str(my_steps_per_epoch)+'_ImageSize'+str(img_size)+'_BatchSize'+str(my_batch_size)
log_path = os.path.join('logs',data_type,'completetrain','')
#model.load_weights('./ModelCheckpoints/26-06-18 16:15__weights.11-2.52.hdf5')

if not os.path.exists(log_path):
    os.makedirs(log_path)

    
model.fit_generator(training_set,
                    steps_per_epoch=my_steps_per_epoch,
                    epochs=my_epochs_retrain, 
                    validation_data=val_set,
                    validation_steps=my_val_steps,
                    shuffle=is_shuffle,
                    #initial_epoch=10,
                    callbacks=[TensorBoard(log_dir=log_path,
                                           write_batch_performance=write_batch_per),
                              ModelCheckpoint('./ModelCheckpoints/'
                                              +time_of_real
                                              +'__weights.{epoch:02d}-val_acc{val_acc:.2f}--train_acc{acc:.2f}.hdf5',
                                              monitor='val_loss',
                                              verbose=1,
                                              save_best_only=False,
                                              save_weights_only=False,
                                              mode='auto',
                                              period=1)
                              ]
                   )

Epoch 1/12
376/376 [==============================] - 591s 2s/step - loss: 0.9420 - acc: 0.7157 - val_loss: 1.1257 - val_acc: 0.7161

Epoch 00001: saving model to ./ModelCheckpoints/05-07-18 22:24__weights.01-val_acc0.72--train_acc0.72.hdf5
Epoch 2/12
376/376 [==============================] - 574s 2s/step - loss: 0.4702 - acc: 0.8528 - val_loss: 1.3752 - val_acc: 0.7066

Epoch 00002: saving model to ./ModelCheckpoints/05-07-18 22:24__weights.02-val_acc0.71--train_acc0.85.hdf5
Epoch 3/12
376/376 [==============================] - 572s 2s/step - loss: 0.2932 - acc: 0.9055 - val_loss: 1.3793 - val_acc: 0.7075

Epoch 00003: saving model to ./ModelCheckpoints/05-07-18 22:24__weights.03-val_acc0.71--train_acc0.91.hdf5
Epoch 4/12
376/376 [==============================] - 571s 2s/step - loss: 0.2288 - acc: 0.9286 - val_loss: 1.6370 - val_acc: 0.6901

Epoch 00004: saving model to ./ModelCheckpoints/05-07-18 22:24__weights.04-val_acc0.69--train_acc0.93.hdf5
Epoch 5/12
376/376 [================

In [20]:
## SAVE THE MODEL
model_path = os.path.join('trainedModels',data_type,'completetrain','')
if not os.path.exists(model_path):
    os.makedirs(model_path)
print('try to save model'+
      model_path+file_name+'.h5')
model.save( model_path+file_name+'.h5')  # creates a HDF5 file 'my_model.h5'
print("Saved model to disk as> "+file_name)

try to save modeltrainedModels/mushrooms24/completetrain/05-07-18 22:24__Epochs12_Steps376_ImageSize299_BatchSize64.h5
Saved model to disk as> 05-07-18 22:24__Epochs12_Steps376_ImageSize299_BatchSize64
